<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:


      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-28 14:11:21
-------------------
qualified stocks: 89
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  2.00 K
Current:  1.45 C
-------------------
Today PnL: -56.49 K (-0.39%)
Current PnL: -19.63 L (-12.87%)
CY Booked + Current PnL: -7.53 L (-4.94%)
-------------------
Total profit:  2.38 L
Total loss:  -22.02 L
-------------------
Total Booked + Current PnL: 19.45 L (15.48%)
Total Booked PnL: 39.08 L (31.11%)
Curr Year Booked PnL: 12.10 L (8.37%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 87.53 L (60.56%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.36%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
79  TTKPRESTIG    770.0   123.69    79.0       M-SC     1.60   96370.0   
59    RELIANCE   1533.0    -7.28    73.0       X-LC     3.84  232034.0   
80  UJJIVANSFB     60.0   132.21    74.0       H-SC    17.53  146367.0   
2   ABBOTINDIA  35195.0    -9.04    45.0       X-MC     2.98   89775.0   
86      VOLTAS   1530.0     1.16    53.0       X-MC     3.54  212340.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
79      -4407.0   4500.0       13.71         -4.37   4.67   0.09    245.0   
59      21668.0   7123.0        0.22         10.30   3.07  13.68     37.0   
80       3888.0  15632.0        0.39          2.73  10.68  13.70    163.0   
2        -903.0  15809.0        0.52         -1.00  17.61  16.44    101.0   
86      20598.0  17157.0       -1.46         10.74   8.08  19.69     99.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
79    -0.98        0.67    26.61    OX40N      NTT    DURABLES  
59     3.04        1.61    28.53     XY25      NTT  REFINERIES  
80     0.25        1.01    72.75    OX40N      NTT       BANKS  
2     -0.06        0.62    18.98      X40      ATH      PHARMA  
86     1.20        1.47    18.27     XY25      NTT          AC

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
79  TTKPRESTIG   770.0   123.69    79.0       M-SC     1.60   96370.0   
39  INDUSINDBK  1800.0   271.15    68.0       L-MC     9.21   45566.0   
84  VALIANTORG   838.0  -290.89    45.0       H-SC     8.68  106284.0   
77    TITAGARH  1548.0    -4.29    52.0       H-SC     2.60  204265.0   
56    RAJOOENG   143.1   -42.59    45.0       H-SC    18.40   87110.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
79      -4407.0    4500.0       13.71         -4.37    4.67   0.09    245.0   
39     -31640.0   57035.0        3.81        -40.98  125.17  32.89    258.0   
84     -57321.0  178632.0        2.59        -35.04  168.07  74.15    139.0   
77     -37912.0  148684.0        2.42        -15.65   72.79  45.74    158.0   
56     -15390.0   55994.0        2.11        -15.01   64.28  39.61    114.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy     Category  
79    -0.98        0.67    26.61    OX40N      NTT     DURABLES  
39    -0.55        0.32    25.48       XR      NTT        BANKS  
84    -0.32        0.74    36.24       XR      NTT    CHEMICALS  
77    -0.25        1.41    32.85     XY24      NTT  ENGINEERING  
56    -0.27        0.60     6.39       AR      ATH         MISC

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
13  BATAINDIA  2096.00   -37.87    36.0       X-SC     4.48   92952.0   
32   HINDZINC   730.22    25.32    44.0       M-LC    11.16  204276.0   
67    SIEMENS  4671.50    -4.11    41.0       H-LC     0.99  154005.0   
60  REPCOHOME   880.00   -56.04    63.0       H-SC     4.02  253102.0   
15      BSOFT   836.99   -14.41    59.0       H-SC    13.96   96390.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
13     -35718.0   81017.0       -3.97        -27.76   87.16  35.20    219.0   
32       -800.0  113373.0       -2.63         -0.39   55.50  54.89     52.0   
67     -32090.0   79574.0       -2.53        -17.24   51.67  25.51     15.0   
60     -33403.0  290738.0       -2.35        -11.66  114.87  89.82    134.0   
15     -44263.0  117046.0       -2.25        -31.47  121.43  51.74    131.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
13    -0.44        0.64     6.69      X40      NTT    FOOTWEAR  
32    -0.01        1.41    23.61      X5K      ATH      METALS  
67    -0.40        1.07    13.49       AR      ATH  ELECTRICAL  
60    -0.11        1.75    33.91     XY24      NTT     FINANCE  
15    -0.38        0.67    11.52       XR      ATH          IT

In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
36       IEX  219.00   -29.93    63.0       H-SC    16.42  200063.0   
57    RECLTD  446.00    42.86    47.0       M-LC     5.50  202620.0   
88     WIPRO  420.00   -14.70    44.0       M-LC     5.79  149742.0   
32  HINDZINC  730.22    25.32    44.0       M-LC    11.16  204276.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
36        109.0   97131.0        0.20          0.05  48.55  48.63    115.0   
57       -550.0   42672.0       -1.42         -0.27  21.06  20.74     55.0   
88      -1203.0  110659.0       -0.98         -0.80  73.90  72.51     53.0   
32       -800.0  113373.0       -2.63         -0.39  55.50  54.89     52.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
36     0.00        1.38    11.61       XR      NTT     MISC  
57    -0.01        1.40     6.57     XY25      NTT  FINANCE  
88    -0.01        1.04     5.18       XR      NTT       IT  
32    -0.01        1.41    23.61      X5K      ATH   METALS

In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
80  UJJIVANSFB    60.0   132.21    74.0       H-SC    17.53  146367.0   
79  TTKPRESTIG   770.0   123.69    79.0       M-SC     1.60   96370.0   
46   KANSAINER   340.0   -67.61    48.0       H-SC     2.57  223047.0   
74   TATAELXSI  9161.0   -22.20    54.0       H-MC     7.05  105156.0   
68         SIS   528.0  2052.85    49.0       H-SC     3.15   86878.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
80       3888.0  15632.0        0.39          2.73  10.68  13.70    163.0   
79      -4407.0   4500.0       13.71         -4.37   4.67   0.09    245.0   
46     -46620.0  82951.0       -2.11        -17.29  37.19  13.47    138.0   
74     -26679.0  68909.0       -0.91        -20.24  65.53  32.03     98.0   
68     -24154.0  47766.0       -0.38        -21.75  54.98  21.26    156.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
80     0.25        1.01    72.75    OX40N      NTT     BANKS  
79    -0.98        0.67    26.61    OX40N      NTT  DURABLES  
46    -0.56        1.54    13.41     XY24      NTT    PAINTS  
74    -0.39        0.73    17.78    OX40N      NTT        IT  
68    -0.51        0.60    17.38    OX40N      NTT      MISC

In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol    FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
12  BANKINDIA  190.0   -15.53    75.0       H-MC    15.23  216812.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
12      37004.0  75407.0         0.9         20.58  34.78  62.52     88.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
12     0.49         1.5    61.68       XR      NTT    BANKS

In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
80  UJJIVANSFB   60.00   132.21    74.0       H-SC    17.53  146367.0   
62        SAIL  228.00    44.35    53.0       M-MC     9.76  227752.0   
47    KPIGREEN  731.05     9.40    69.0       H-SC     5.28  130372.0   
81   UNIONBANK  163.00    -8.53    65.0       M-LC     9.82  166229.0   
12   BANKINDIA  190.00   -15.53    75.0       H-MC    15.23  216812.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
80       3888.0   15632.0        0.39          2.73  10.68  13.70    163.0   
62       2790.0  164870.0        1.82          1.24  72.39  74.53    192.0   
47       5075.0   53857.0       -0.93          4.05  41.31  47.03    141.0   
81      25389.0   19100.0        0.86         18.03  11.49  31.59     66.0   
12      37004.0   75407.0        0.90         20.58  34.78  62.52     88.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
80     0.25        1.01    72.75    OX40N      NTT    BANKS  
62     0.02        1.58    33.89     XY24      BTT    STEEL  
47     0.09        0.90    63.24       MH      ATH    POWER  
81     1.33        1.15    49.29     XY24      NTT    BANKS  
12     0.49        1.50    61.68       XR      NTT    BANKS

In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
12   BANKINDIA   190.00   -15.53    75.0       H-MC    15.23  216812.0   
80  UJJIVANSFB    60.00   132.21    74.0       H-SC    17.53  146367.0   
87   WHIRLPOOL  2270.00   -38.77    63.0       M-SC     3.19  126969.0   
47    KPIGREEN   731.05     9.40    69.0       H-SC     5.28  130372.0   
37   INDIAMART  4810.62   -53.15    58.0       H-SC     6.90  130915.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  CumlRnk  \
12      37004.0   75407.0        0.90         20.58  34.78   62.52     88.0   
80       3888.0   15632.0        0.39          2.73  10.68   13.70    163.0   
87      12171.0   81870.0       -0.80         10.60  64.48   81.92    223.0   
47       5075.0   53857.0       -0.93          4.05  41.31   47.03    141.0   
37       7579.0  124042.0        0.61          6.15  94.75  106.72    119.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
12     0.49        1.50    61.68       XR      NTT     BANKS  
80     0.25        1.01    72.75    OX40N      NTT     BANKS  
87     0.15        0.88    50.31       XR      NTT  DURABLES  
47     0.09        0.90    63.24       MH      ATH     POWER  
37     0.06        0.91    30.96       AR      ATH      MISC

In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
75  TATAMOTORS   1065.00   -53.17    19.0       X-LC    16.99  148848.0   
24       DMART   5391.45   -17.39    35.0       X-LC     9.99  100992.0   
45    JSWINFRA    342.00   -20.69    36.0       X-MC     5.38  199091.0   
13   BATAINDIA   2096.00   -37.87    36.0       X-SC     4.48   92952.0   
53        PGHH  17907.65   -30.81    36.0       X-MC     2.88  200310.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
75    -127675.0  237740.0        1.67        -46.17  159.72  39.81     54.0   
24      -1594.0   28399.0       -1.17         -1.55   28.12  26.13     38.0   
45      -1256.0   31755.0       -0.35         -0.63   15.95  15.23    178.0   
13     -35718.0   81017.0       -3.97        -27.76   87.16  35.20    219.0   
53       -510.0   68306.0       -0.40         -0.25   34.10  33.76     80.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
75    -0.54        1.03     4.91     XY24      NTT      AUTO  
24    -0.06        0.70    23.62     X40N      ATH      FMCG  
45    -0.04        1.38    31.42     X40N      NTT    REALTY  
13    -0.44        0.64     6.69      X40      NTT  FOOTWEAR  
53    -0.01        1.39     4.93      X40      ATH      FMCG

In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
67  SIEMENS   4671.50    -4.11    41.0       H-LC     0.99  154005.0   
42      ITC    452.00   -37.08    61.0       X-LC     1.56  203717.0   
53     PGHH  17907.65   -30.81    36.0       X-MC     2.88  200310.0   
29  HAVELLS   2069.16    -6.89    42.0       X-MC     3.09  310800.0   
86   VOLTAS   1530.00     1.16    53.0       X-MC     3.54  212340.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
67     -32090.0   79574.0       -2.53        -17.24  51.67  25.51     15.0   
42       3579.0   17316.0       -0.96          1.79   8.50  10.44      4.0   
53       -510.0   68306.0       -0.40         -0.25  34.10  33.76     80.0   
29     -17191.0  123729.0       -0.84         -5.24  39.81  32.48     92.0   
86      20598.0   17157.0       -1.46         10.74   8.08  19.69     99.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
67    -0.40        1.07    13.49       AR      ATH  ELECTRICAL  
42     0.21        1.41     7.45      X40      NTT        FMCG  
53    -0.01        1.39     4.93      X40      ATH        FMCG  
29    -0.14        2.15     5.59      X40      ATH  ELECTRICAL  
86     1.20        1.47    18.27     XY25      NTT          AC

In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
10  BAJAJHFL    181.50   -15.33    41.0       X-MC     5.88  178432.0   
21    COLPAL   3767.14    -4.54    45.0       X-MC     8.06  222730.0   
52   PAGEIND  51769.93   -27.66    38.0       X-MC     6.87   81670.0   
3        ACC   3906.00   -37.64    50.0       X-MC     2.70  185250.0   
85       VBL    671.64   -19.59    47.0       X-LC     9.52  289812.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
10     -22074.0  119228.0       -0.93        -11.01   66.82  48.45     90.0   
21     -40635.0  153973.0        0.49        -15.43   69.13  43.04     84.0   
52      -1840.0   21871.0       -0.37         -2.20   26.78  23.98     82.0   
3      -52701.0  205350.0       -0.55        -22.15  110.85  64.15    174.0   
85     -26030.0  138704.0       -1.11         -8.24   47.86  35.67      5.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
10    -0.19        1.23     2.89     X40N      ATH   FINANCE  
21    -0.26        1.54     3.40     XY25      ATH      FMCG  
52    -0.08        0.57     3.49      X40      ATH  APPARELS  
3     -0.26        1.28     3.92     XY24      BTT    CEMENT  
85    -0.19        2.01     4.47     X40N      ATH      FMCG

In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%  Current  \
54       QUESS    986.00   -45.45    50.0       X-SC    38.37  54093.0   
58      RELAXO   1176.00   -40.27    41.0       X-SC     4.69  81442.0   
52     PAGEIND  51769.93   -27.66    38.0       X-MC     6.87  81670.0   
2   ABBOTINDIA  35195.00    -9.04    45.0       X-MC     2.98  89775.0   
13   BATAINDIA   2096.00   -37.87    36.0       X-SC     4.48  92952.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
54     -10913.0  161841.0        1.54        -16.79  299.19  232.18    198.0   
58     -63718.0  139649.0       -0.40        -43.90  171.47   52.31    136.0   
52      -1840.0   21871.0       -0.37         -2.20   26.78   23.98     82.0   
2        -903.0   15809.0        0.52         -1.00   17.61   16.44    101.0   
13     -35718.0   81017.0       -3.97        -27.76   87.16   35.20    219.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
54    -0.07        0.37     5.51     XY24      NTT      MISC  
58    -0.46        0.56     9.01     X40N      NTT  FOOTWEAR  
52    -0.08        0.57     3.49      X40      ATH  APPARELS  
2     -0.06        0.62    18.98      X40      ATH    PHARMA  
13    -0.44        0.64     6.69      X40      NTT  FOOTWEAR

In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
76    TCS  4389.97   -28.04    53.0       X-LC    11.69  286258.0   
40   INFY  2275.00   -18.52    56.0       X-LC     6.58  318030.0   
42    ITC   452.00   -37.08    61.0       X-LC     1.56  203717.0   
85    VBL   671.64   -19.59    47.0       X-LC     9.52  289812.0   
1     ABB  7934.00   -40.84    45.0       H-LC     6.00  247296.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
76     -59698.0  126412.0       -1.28        -17.26  44.16  19.28      1.0   
40       5142.0  166552.0       -0.76          1.64  52.37  54.87      3.0   
42       3579.0   17316.0       -0.96          1.79   8.50  10.44      4.0   
85     -26030.0  138704.0       -1.11         -8.24  47.86  35.67      5.0   
1      -14323.0  133540.0       -1.64         -5.47  54.00  45.57      7.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
76    -0.47        1.98     5.83      X40      ATH          IT  
40     0.03        2.20    10.06      X40      BTT          IT  
42     0.21        1.41     7.45      X40      NTT        FMCG  
85    -0.19        2.01     4.47     X40N      ATH        FMCG  
1     -0.11        1.71     5.23       AR      NTT  ELECTRICAL

In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
7   ASIANTILES   137.00  7155.56    64.0       L-SC     7.47   79579.0   
20   COFFEEDAY    80.00   -43.66    51.0       L-SC    27.68   82080.0   
50      MASFIN   398.61   -20.51    45.0       H-SC     9.13   90540.0   
79  TTKPRESTIG   770.00   123.69    79.0       M-SC     1.60   96370.0   
87   WHIRLPOOL  2270.00   -38.77    63.0       M-SC     3.19  126969.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
7      -14231.0  90847.0        1.12        -15.17  114.16  81.67    269.0   
20     -31469.0  71516.0        1.28        -27.71   87.13  35.27    268.0   
50      -7440.0  29045.0        0.13         -7.59   32.08  22.05    152.0   
79      -4407.0   4500.0       13.71         -4.37    4.67   0.09    245.0   
87      12171.0  81870.0       -0.80         10.60   64.48  81.92    223.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
7     -0.16        0.55    54.11       XR      NTT  CERAMICS  
20    -0.44        0.57   100.89       XR      NTT    HOTELS  
50    -0.26        0.63    31.61       XR      ATH   FINANCE  
79    -0.98        0.67    26.61    OX40N      NTT  DURABLES  
87     0.15        0.88    50.31       XR      NTT  DURABLES

In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
63  SAMMAANCAP   326.00  -197.73    79.0       M-SC    28.51  168858.0   
69    SONACOMS   806.63   -32.20    70.0       M-SC     8.30   84402.0   
12   BANKINDIA   190.00   -15.53    75.0       H-MC    15.23  216812.0   
5     ANGELONE  3033.00    16.84    66.0       X-SC     5.33  206763.0   
80  UJJIVANSFB    60.00   132.21    74.0       H-SC    17.53  146367.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
63      19638.0  124550.0       -0.54         13.16  73.76  96.62    208.0   
69     -16757.0   56760.0       -0.32        -16.56  67.25  39.54    202.0   
12      37004.0   75407.0        0.90         20.58  34.78  62.52     88.0   
5       15757.0   41952.0       -2.15          8.25  20.29  30.21    157.0   
80       3888.0   15632.0        0.39          2.73  10.68  13.70    163.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
63     0.16        1.17    82.74     XY25      NTT  FINANCE  
69    -0.30        0.58    19.02       AR      ATH     AUTO  
12     0.49        1.50    61.68       XR      NTT    BANKS  
5      0.38        1.43    30.85     X40N      NTT  FINANCE  
80     0.25        1.01    72.75    OX40N      NTT    BANKS

In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.34
1     25       43.44
2     50       74.07

In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.85
LC    31.84
MC    24.34
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.81
X40      16.10
XY25     11.96
XR       11.33
X40N     10.46
AR        7.71
OX40N     7.54
X200      1.74
X5K       1.41
SR        1.07
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.25
X-LC    21.32
X-MC    18.57
M-SC    12.61
M-LC     5.00
X-SC     4.54
H-LC     4.52
H-MC     3.87
M-MC     1.58
L-SC     1.45
L-LC     1.00
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                14.27         -4.54   38.65
IT                  12.37        -17.73   80.88
FINANCE             12.01         -8.87   58.22
MISC                 6.82        -16.34   80.81
BANKS                6.68         -3.44   57.86
ELECTRICAL           5.70         -9.83   48.76
PAINTS               5.44        -17.38   34.88
INSURANCE            3.83         -0.03   37.33
AUTO                 2.75        -44.62  104.97
DURABLES             2.36        -13.44   41.04
FOOTWEAR             2.30        -32.39   85.47
TRAVEL               2.18        -42.43  113.75
AC                   2.14         -3.60   15.83
CERAMICS             2.13        -30.39   85.65
BREWERIES            1.61         -0.83   21.77
REFINERIES           1.61          9.34    3.07
STEEL                1.58          1.23   72.39
METALS               1.41         -0.39   55.50
ENGINEERING          1.41        -18.56   72.79
REALTY               1.38         -0.63   15.95
CHEMICALS            1.30        -41.25  156.49
CEMENT               1.28        -28.45  110.85
JEWELLERY            1.24        -79.03  141.23
HEALTHCARE           1.15         -8.53   35.77
MINING               1.00         -4.10   27.47
ENTERTAINMENT        0.93        -42.98  128.25
POWER                0.90          3.89   41.31
PHARMA               0.62         -1.01   17.61
HOTELS               0.57        -38.34   87.13
APPARELS             0.57         -2.25   26.78
TEXTILES             0.49        -31.14   68.54

In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3264213.0     22
XR        1314889.0     14
AR        1174492.0      9
X40        831714.0     12
X40N       648825.0      9
OX40N      557688.0     10
XY25       454517.0      8
SR         266817.0      2
X5K        113373.0      1
X200        72759.0      1
MH          53857.0      1

In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        3345027.0     25
M-SC        1222325.0     15
X-MC        1195001.0     14
X-LC        1060438.0     13
X-SC         523043.0      6
H-MC         321627.0      3
H-LC         285873.0      3
M-LC         285804.0      4
L-SC         252396.0      3
M-MC         164870.0      1
L-MC          57035.0      1
L-LC          39705.0      1

In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1148591.0      6
           AR         837353.0      5
           XR         779340.0      7
M-SC       XY24       703988.0      6
X-MC       XY24       581968.0      4
X-LC       X40        459777.0      5
           XY24       306544.0      2
X-MC       X40        290920.0      6
X-SC       X40N       280185.0      4
H-SC       SR         266817.0      2
           OX40N      259069.0      4
X-LC       X40N       217657.0      3
H-LC       AR         213114.0      2
H-MC       XY24       177311.0      1
X-MC       XY25       171130.0      2
M-MC       XY24       164870.0      1
L-SC       XR         162363.0      2
X-SC       XY24       161841.0      1
X-MC       X40N       150983.0      2
M-SC       OX40N      139677.0      4
           XR         130085.0      2
           XY25       124550.0      1
           AR         124025.0      2
M-LC       X5K        113373.0      1
           XR         110659.0      1
L-SC       OX40N       90033.0      1
X-SC       X40         81017.0      1
X-LC       XY25        76460.0      3
H-MC       XR          75407.0      1
H-LC       X200        72759.0      1
H-MC       OX40N       68909.0      1
L-MC       XR          57035.0      1
H-SC       MH          53857.0      1
M-LC       XY25        42672.0      1
L-LC       XY25        39705.0      1
M-LC       XY24        19100.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
